# Heart Disease Classification with Neural Networks

## Setup
### Import all necessary libraries and define constants.

In [1]:
import numpy as np
from dataset import MyDataClass
from preprocess import preprocess_heart_disease_from_csv
from models import Layer_Dense, Activation_ReLU, Activation_Softmax, Loss_CategoricalCrossentropy, Accuracy_Categorical, Optimizer_Adam, Model

## Data Preprocessing
### Load and preprocess the data. This section should end with the dataset ready for training.

In [2]:
preprocess_heart_disease_from_csv()

In [3]:
data = MyDataClass()
data.load_and_preprocess_heart_disease()


In [4]:
model = Model()

model.add(Layer_Dense(data.X_train.shape[1], 6))
model.add(Activation_ReLU())
model.add(Layer_Dense(6, 6, weight_regularizer_l2=5e-3, bias_regularizer_l2=5e-3))
model.add(Activation_ReLU())
model.add(Layer_Dense(6, 5))
model.add(Activation_Softmax())
model.set(
    loss=Loss_CategoricalCrossentropy(),
    optimizer=Optimizer_Adam(learning_rate=1e-4, decay=5e-7),
    accuracy=Accuracy_Categorical()
)

model.finalize()



In [5]:
model.train(data.X_train, data.y_train, validation_data=(data.X_test, data.y_test), epochs=2000, batch_size=16, print_every=100)

ZeroDivisionError: division by zero

In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curves(model):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(model.train_acc_history, label='Training Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(model.train_loss_history, label='Training Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_learning_curves(model)


## Classification report

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(data.X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = data.y_test

report = classification_report(y_true, y_pred_classes, target_names=[str(i) for i in range(5)], zero_division=0)
print(report)


## Latent features visualization

In [ ]:
example_indices = np.random.choice(range(len(data.X_test)), 10)
examples = data.X_test[example_indices]

activations = model.get_activations(examples)


In [ ]:
import seaborn as sns

for i, activation in enumerate(activations):
    plt.figure(figsize=(10, 1))
    sns.heatmap(activation, cmap="viridis", yticklabels=False)
    plt.title(f"Layer {i+1} Activation")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

weights = model.get_weights()
biases = model.get_biases()

num_layers = len(weights)

fig, axes = plt.subplots(num_layers, 2, figsize=(12, num_layers * 4))

for i in range(num_layers):
    ax = axes[i, 0]
    sns.heatmap(weights[i], ax=ax, cmap="viridis")
    ax.set_title(f'Layer {i+1} Weights')

    ax = axes[i, 1]
    sns.heatmap(biases[i].reshape(1, -1), ax=ax, cmap="viridis")
    ax.set_title(f'Layer {i+1} Biases')

plt.tight_layout()
plt.show()

## Hyperparameter search using grid search

In [ ]:
learning_rates = [1e-3, 1e-4, 1e-5]
batch_sizes = [8, 16, 32]
epochs_options = [10, 100, 1000]
regularizer_options = [1e-2, 1e-3, 1e-4]

best_hyperparams = None
best_accuracy = 0

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for epochs in epochs_options:
            for regularizer in regularizer_options:
                # Initialize and compile model
                model = Model()
                model.add(Layer_Dense(data.X_train.shape[1], 6))
                model.add(Activation_ReLU())
                model.add(Layer_Dense(6, 6, weight_regularizer_l2=regularizer, bias_regularizer_l2=regularizer))
                model.add(Activation_ReLU())
                model.add(Layer_Dense(6, 5))
                model.add(Activation_Softmax())
                model.set(
                    loss=Loss_CategoricalCrossentropy(),
                    optimizer=Optimizer_Adam(learning_rate=learning_rate),
                    accuracy=Accuracy_Categorical()
                )
                model.finalize()
    
                model.train(data.X_train, data.y_train, validation_data=(data.X_test, data.y_test), epochs=epochs, batch_size=batch_size)
    
                validation_loss, validation_accuracy = model.evaluate(data.X_test, data.y_test)
    
                if validation_accuracy > best_accuracy:
                    best_accuracy = validation_accuracy
                    best_hyperparams = {'learning_rate': learning_rate, 'batch_size': batch_size, 'epochs': epochs, 'regularizer': regularizer}

print("Best Hyperparameters:")
print(best_hyperparams)

## Measuring impact of neural network dimensionality

In [ ]:
learning_rate = 0.0001
batch_size = 32
epochs = 1000
regularizer = 0.0001

layer_options = [1, 2, 3]
layer_results = []

for num_layers in layer_options:
    model = Model()

    model.add(Layer_Dense(data.X_train.shape[1], 6))
    model.add(Activation_ReLU())

    for _ in range(num_layers - 1):
        model.add(Layer_Dense(6, 6, weight_regularizer_l2=regularizer, bias_regularizer_l2=regularizer))
        model.add(Activation_ReLU())

    model.add(Layer_Dense(6, 5))
    model.add(Activation_Softmax())

    model.set(
        loss=Loss_CategoricalCrossentropy(),
        optimizer=Optimizer_Adam(learning_rate=learning_rate),
        accuracy=Accuracy_Categorical()
    )
    model.finalize()

    model.train(data.X_train, data.y_train, validation_data=(data.X_test, data.y_test), epochs=epochs, batch_size=batch_size)

    validation_loss, validation_accuracy = model.evaluate(data.X_test, data.y_test)

    layer_results.append({
        'num_layers': num_layers,
        'loss': validation_loss,
        'accuracy': validation_accuracy
    })
    

In [ ]:
num_layers_list = [result['num_layers'] for result in layer_results]
losses = [result['loss'] for result in layer_results]
accuracies = [result['accuracy'] for result in layer_results]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(num_layers_list, losses, marker='o')
plt.title('Validation Loss vs. Number of Layers')
plt.xlabel('Number of Layers')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(num_layers_list, accuracies, marker='o')
plt.title('Validation Accuracy vs. Number of Layers')
plt.xlabel('Number of Layers')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()